## Generate Tezos Keys from Ledger Nano S Mnemonic Phrase

This is a test to figure out how the ledger nano S generates key hashes from recovery phrases.

In [8]:
phrase = 'motor jazz team food when coach reward hidden obtain faculty tornado crew toast inhale purchase conduct cube omit illness carbon ripple thank crew material'
pkh = 'tz1MN2TrBeSeTvj7Lu3RxeqRwvTcBBo98iza'

In [18]:
from mnemonic import Mnemonic
import hashlib as hl
import numpy as np
import pysodium

In [23]:
mnemo = Mnemonic('english')
entropy = mnemo.to_entropy(phrase)
seed = Mnemonic.to_seed(phrase)
print(entropy.hex())
print(seed.hex())

906eef7aad6fa2596e2b5b988a379599ae30e76b897635734dc4913ba5bf8cd4
accecb88d83f6fe670361e31dcf853f688d1b3f7882e9e5e3367b66e637eb15b8a80066eb1b5b495bd55c48fd30717b5a877a66192f05a9f64cd50b38c41e7eb


In [34]:
# mnemonic phrase to seed and secret exponent
password = ''
mnemonic = my_phrase
passphrase = 'mnemonic' + password
# passphrase = ''
mnemonic_bytes = mnemonic.encode("utf-8")
print(mnemonic_bytes)
passphrase_bytes = passphrase.encode("utf-8")
stretched = hl.pbkdf2_hmac("sha512", mnemonic_bytes, passphrase_bytes, 2048)
seed = stretched[:64]
print(seed.hex())
# for P256/secp256r1 secret exponent is the first 32 bytes of the seed
secret_exponent = seed[:32]
print(secret_exponent.hex())

b'motor jazz team food when coach reward hidden obtain faculty tornado crew toast inhale purchase conduct cube omit illness carbon ripple thank crew material'
accecb88d83f6fe670361e31dcf853f688d1b3f7882e9e5e3367b66e637eb15b8a80066eb1b5b495bd55c48fd30717b5a877a66192f05a9f64cd50b38c41e7eb
accecb88d83f6fe670361e31dcf853f688d1b3f7882e9e5e3367b66e637eb15b


In [35]:
public_point, secret_exponent = pysodium.crypto_sign_seed_keypair(seed=secret_exponent)
print(public_point.hex())
print(secret_exponent.hex())


4bf723052e2f624cf23b4dee2d7b00715e352b86cbbb27f2f286bc7f5e302a4b
accecb88d83f6fe670361e31dcf853f688d1b3f7882e9e5e3367b66e637eb15b4bf723052e2f624cf23b4dee2d7b00715e352b86cbbb27f2f286bc7f5e302a4b


In [38]:
from pyblake2 import blake2b
from pytezos.crypto.encoding import base58_decode, base58_encode
# generate hash
pkh = blake2b(data=public_point, digest_size=20).digest()
print(pkh.hex())
prefix = b'tz1'
public_hash = base58_encode(pkh, prefix).decode()
print(public_hash)



ImportError: cannot import name 'ParserError' from 'dateutil.parser._parser' (/usr/lib/python3/dist-packages/dateutil/parser/_parser.py)

In [20]:
# add checksum
h = hl.sha256()
h.update(entropy)
entropy_cs = entropy
entropy_cs.append(h.digest()[0])
print(entropy_cs.hex())

906eef7aad6fa2596e2b5b988a379599ae30e76b897635734dc4913ba5bf8cd447


In [21]:
# convert byte array to bit array
bits = []
for byte in entropy_cs:
    for i in range(8):
        if (byte & 0b10000000 >> i):
            bits.append(1)
        else:
            bits.append(0)
        
print(bits)
print(len(bits))

# import vocab
# import the vocabulary
f = open('bip39english.txt', 'r')
vocab = [line.strip('\n') for line in f]
f.close()

# iterate through bits with 11 bit chunks
my_words = []
for k in range(0,int(len(bits)/11)):
    chunk = bits[k*11:k*11+11]
    index = 0
    for j in range(11):
        index += chunk[j]*2**(11-1-j)
    my_words.append(vocab[index])
my_phrase = ' '.join(my_words)
print(my_phrase)

[1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1]
264
motor jazz team food when coach reward hidden obtain faculty tornado crew toast inhale purchase conduct cube omit illness carbon ripple thank crew material


In [28]:
# generate public point from secret
public_point, secret_exponent = pysodium.crypto_sign_seed_keypair(seed=seed[:32])
print(secret_exponent.hex())
print(public_point.hex())

accecb88d83f6fe670361e31dcf853f688d1b3f7882e9e5e3367b66e637eb15b4bf723052e2f624cf23b4dee2d7b00715e352b86cbbb27f2f286bc7f5e302a4b
4bf723052e2f624cf23b4dee2d7b00715e352b86cbbb27f2f286bc7f5e302a4b


NameError: name 'blake2b' is not defined

In [ ]:
mnemo = Mnemonic("english")
phrase = mnemo.generate(256)
print(phrase)

In [16]:
entropy = mnemo.to_entropy(phrase)
entropy_list = list(entropy)
print(entropy)
print(entropy_list)
print(entropy.hex())

bytearray(b"\x95\xcb\x7f\xab\xf4\xec\xb8\xbf\xcd\x81\xc3\x9d\xd1\xe4\xe2\xed\xc7\xe0v\xfey\xe4\'\xd3.G\x80\x93#\x92i\xb5")
[149, 203, 127, 171, 244, 236, 184, 191, 205, 129, 195, 157, 209, 228, 226, 237, 199, 224, 118, 254, 121, 228, 39, 211, 46, 71, 128, 147, 35, 146, 105, 181]
95cb7fabf4ecb8bfcd81c39dd1e4e2edc7e076fe79e427d32e478093239269b5


In [17]:
# calculate checksum length
el = len(entropy)*8 # length in bits
csl = int(el/32) # checksum length
print(csl)

8


In [18]:
h = hl.sha256()
h.update(entropy)
entropy.append(h.digest()[0])
print(entropy.hex())

95cb7fabf4ecb8bfcd81c39dd1e4e2edc7e076fe79e427d32e478093239269b5bf


In [19]:
# convert byte array to bit array
bits = []
for byte in entropy:
    for i in range(8):
        if (byte & 0b10000000 >> i):
            bits.append(1)
        else:
            bits.append(0)
        
print(bits)
print(len(bits))

[1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1]
264


In [20]:
# import vocab
# import the vocabulary
f = open('bip39english.txt', 'r')
vocab = [line.strip('\n') for line in f]
f.close()

In [21]:
# iterate through bits with 11 bit chunks
my_words = []
for k in range(0,int(len(bits)/11)):
    chunk = bits[k*11:k*11+11]
    index = 0
    for j in range(11):
        index += chunk[j]*2**(11-1-j)
    my_words.append(vocab[index])
my_phrase = ' '.join(my_words)
print(my_phrase)

noise fossil turtle truth slide garment current debris oven monkey december swear lawn derive wheat develop chief offer music abuse cram since plug retire


## Entropy to Mnemonic

The following shows how to go from a random seed (entropy) to 24-word mnemonic phrase

In [32]:
# generate 256 bit random number
myentropy = bytearray(np.random.bytes(32))
print(myentropy.hex())

aba66986f3cf0c4c8364921dbb0feddaa8b4a542c96daf48bcc50f367dcb14c9


In [33]:
# add hash byte
h = hl.sha256()
h.update(myentropy)
myentropy.append(h.digest()[0])
print(myentropy.hex())

aba66986f3cf0c4c8364921dbb0feddaa8b4a542c96daf48bcc50f367dcb14c980


In [37]:
# convert byte array to bit array
bits = []
for byte in myentropy:
    for i in range(8):
        if (byte & 0b10000000 >> i):
            bits.append(1)
        else:
            bits.append(0)
        
print(bits)
print(len(bits))

[1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]
264


In [38]:
# import vocab
# import the vocabulary
f = open('bip39english.txt', 'r')
vocab = [line.strip('\n') for line in f]
f.close()

In [39]:
# iterate through bits with 11 bit chunks
my_words = []
for k in range(0,int(len(bits)/11)):
    chunk = bits[k*11:k*11+11]
    index = 0
    for j in range(11):
        index += chunk[j]*2**(11-1-j)
    my_words.append(vocab[index])
my_phrase = ' '.join(my_words)
print(my_phrase)

produce crew giant travel valid chaos assault myself buffalo such window relief mercy famous arctic color quantum echo course bunker guitar tortoise farm copy


In [40]:
# check
e = mnemo.to_entropy(my_phrase)
e_list = list(e)
print(e)
print(e_list)
print(e.hex())

bytearray(b'\xab\xa6i\x86\xf3\xcf\x0cL\x83d\x92\x1d\xbb\x0f\xed\xda\xa8\xb4\xa5B\xc9m\xafH\xbc\xc5\x0f6}\xcb\x14\xc9')
[171, 166, 105, 134, 243, 207, 12, 76, 131, 100, 146, 29, 187, 15, 237, 218, 168, 180, 165, 66, 201, 109, 175, 72, 188, 197, 15, 54, 125, 203, 20, 201]
aba66986f3cf0c4c8364921dbb0feddaa8b4a542c96daf48bcc50f367dcb14c9


## Mnemonic to Seed

Tezos takes the mnemonic and hashes it with an optional passphrase to formulate a secret key.

In [43]:
password = ''
mnemonic = my_phrase
passphrase = 'mnemonic' + password
mnemonic_bytes = mnemonic.encode("utf-8")
print(mnemonic_bytes)
passphrase_bytes = passphrase.encode("utf-8")
stretched = hl.pbkdf2_hmac("sha512", mnemonic_bytes, passphrase_bytes, 2048)
seed = stretched[:64]
print(seed.hex())
# for P256/secp256r1 secret exponent is the first 32 bytes of the seed
secret_exponent = seed[:32]
print(secret_exponent.hex())

b'produce crew giant travel valid chaos assault myself buffalo such window relief mercy famous arctic color quantum echo course bunker guitar tortoise farm copy'
b313d7649ff4468c7291fa8bc289d55ea98f327b02a569538290ff2ffa1142049623a9f439ab7c87c747b7635bc4034806986339048ae2f1cf3a745198e93095
b313d7649ff4468c7291fa8bc289d55ea98f327b02a569538290ff2ffa114204


In [48]:
# P256
import fastecdsa.keys 
import fastecdsa.curve
from fastecdsa.encoding.util import bytes_to_int
import fastecdsa.encoding.sec1
from pyblake2 import blake2b
from pytezos.crypto.encoding import base58_decode, base58_encode

pk = fastecdsa.keys.get_public_key(bytes_to_int(secret_exponent), curve=fastecdsa.curve.P256)
public_point = fastecdsa.encoding.sec1.SEC1Encoder.encode_public_key(pk)
print(pk)
print(public_point.hex())

X: 0x34e4e30d7824347ba7ea79c8ebf2d71531ac0ff200e2732b8535e20bc8c6cabe
Y: 0x99b7582413d274775f704fa826fdbe4e9544388f80ee205f0d5de0d2d1b522dd
(On curve <P256>)
0334e4e30d7824347ba7ea79c8ebf2d71531ac0ff200e2732b8535e20bc8c6cabe


In [47]:
pkh = blake2b(data=public_point, digest_size=20).digest()
print(pkh.hex())
prefix = b'tz3'
public_hash = base58_encode(pkh, prefix).decode()
print(public_hash)

394dd12312883880e4ade8d631be039d99f4020b
tz3RZ3MQsNNHCbPtohBZh4wGAMJH9BgTKV1m


In [51]:
# compare to pytezos results
from pytezos.crypto.key import Key
p2pk = Key.from_public_point(
    bytes.fromhex(public_point.hex()), 
    curve=b'p2')
p2pk


Public key hash
tz3RZ3MQsNNHCbPtohBZh4wGAMJH9BgTKV1m

Helpers
.blinded_public_key_hash()
.from_alias()
.from_encoded_key()
.from_faucet()
.from_mnemonic()
.from_public_point()
.from_secret_exponent()
.generate()
.public_key()
.public_key_hash()
.secret_key()
.sign()
.verify()

### Questions

How does a ledger convert seed phrase into keys and key hash? 